# Build a simple image classifier from a standard dataset
Adapted from https://www.tensorflow.org/tutorials/images/cnn

This tutorial demonstrates training a simple [Convolutional Neural Network](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN) to classify [CIFAR images](https://www.cs.toronto.edu/~kriz/cifar.html). Because this tutorial uses the [Keras Sequential API](https://www.tensorflow.org/guide/keras/overview), creating and training your model will take just a few lines of code.


### Import TensorFlow

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Download and prepare the CIFAR10 dataset


The CIFAR10 dataset contains 60,000 color images in 10 classes, with 6,000 images in each class. The dataset is divided into 50,000 training images and 10,000 testing images. The classes are mutually exclusive and there is no overlap between them.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Verify the data

To verify that the dataset looks correct, let's plot the first 25 images from the training set and display the class name below each image:


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Create the neural network

We're going to skip over the details for now of how this works. We'll talk about this in lecture next week. If you want more information now, though, feel free to refer to the [Original tutorial](https://www.tensorflow.org/tutorials/images/cnn). 

For now, trust that the code below makes a new neural network in keras. Each `.add()` function call either adds a new layer of neurons of a particular type, or specifies some processing that will be applied to the neurons in the layer that was just added.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

### Compile and train the model

The `compile` function specifies which optimisation algorithm to use when training the network. Here, we are using the "adam" optimizer and we are telling it to minimise a particular type of error (loss) called "sparse categorical cross-entropy" (more on these later; don't worry for now). The `metrics=['accuracy']` bit tells it to print out the model accuracy on the training and test datasets after every epoch (training pass). Validation set should generally improve as you train for more epochs, though sometimes the model begins to *overfit* and you will see validation set accuracy decrease even if training set accuracy continues to be good.

The `fit` function actually does the model training. The code below specifies that 10 training epochs should be performed. Note that we pass the "validation data" (i.e., our hold-out set) to the `fit` function only for the purposes of computing the accuracy on this dataset after each epoch, which we'll examine later, below.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### Plot how accuracy changes over time on the training set and the test ("validation") set

In [ ]:
plt.plot(history.history['accuracy'], label='training set accuracy')
plt.plot(history.history['val_accuracy'], label = 'test set accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
#You can use code like this to test a trained model on any other dataset
# Here we're using test_images and test_labels again, but you could imagine replacing this with totally different data
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Test accuracy after final epoch is ", test_acc*100)